In [ ]:
%config Completer.use_jedi = False
import sys
sys.path.append('../')

In [ ]:
from checklist.test_suite import TestSuite
import pandas as pd

pd.set_option('display.max_colwidth', None)

def load_suite(path):
    suite = TestSuite.from_file(path)
    summary_table = suite.visual_summary_table()
    
    return suite, summary_table


def proccess_summary_do_dataframe(suite, summary_table):
    test_templates = summary_table.get_state()['test_infos']

    items = []
    templates = []
    
    for test_template in test_templates:
        test_template_name = test_template['name']
        template_name = test_template_name.split(' - ')[1]
        test_cases = suite.visual_summary_by_test(test_template_name).filtered_testcases

        templates.append(template_name)

        for item in test_cases:
            text = item['examples'][0]['new']['text']
            pred = item['examples'][0]['new']['pred']
            label = item['examples'][0]['label']
            succeed = item['examples'][0]['succeed']

            items.append([text, label, pred, succeed, template_name])

    df = pd.DataFrame(items, columns=['text', 'label', 'pred', 'succeed', 'template'])
    
    return df, templates
    
    
def sample_df(df_orig, percent):
    total = ceil(percent * len(df_orig))
    
    return df_orig.sample(total)    

In [ ]:
from math import ceil

!rmdir /s /q test_cases
!mkdir test_cases

percent = 0.1
approachs = ['approach1','approach2','approach3','approach4','approach5','random']

for appr in approachs:
    suite, tb = load_suite(f'./suites/posneg-{appr}.suite')
    df, templates = proccess_summary_do_dataframe(suite, tb)

    print(f'Proccessing {appr}...')
    filename = f'test_cases/{appr}.xlsx'
    with pd.ExcelWriter(filename) as writer:  
        for template in templates:
            df_filtered = df[df['template'] == template]
            df_filtered_0 = sample_df(df_filtered[df['succeed'] == 0], percent)
            df_filtered_1 = sample_df(df_filtered[df['succeed'] == 1], percent)

            df_filtered = pd.concat([df_filtered_0, df_filtered_1], axis=0)

            columns = ['text', 'label', 'pred', 'succeed']

            df_filtered.to_excel(writer, sheet_name=template, index=False, columns=columns)
